# Cross Encoder 重排器

## 概述
Cross Encoder 重排器用於 **RAG（檢索增強生成）** 系統中，將初步檢索出的候選文件重新排序，讓最相關的內容排在前面，提升最終生成結果的精準度。

---

## 與 Bi-Encoder 的差別
- **Bi-Encoder**：分別編碼查詢與文件，比對向量相似度，速度快但精度較低。
- **Cross Encoder**：將查詢與文件組合成單一輸入，進行深度語意分析，精度高但計算較慢。

---

## 工作方式
1. **向量檢索**：先用向量檢索（如 FAISS）快速抓取候選文件（例如 50 筆）。
2. **交互編碼**：Cross Encoder 同時處理查詢與每份文件，輸出相關性分數（0–1）。
3. **重新排序**：依分數高低重新排列，保留前 N 筆給生成模型使用。

---

## 優點
- **精準度高**：理解查詢與文件的細節關係。
- **歧義處理佳**：對多義詞、上下文有更好判斷。
- **顯著提升相關性**：檢索效果通常可提升 10–30%。

---

## 缺點
- **計算較慢**：比純向量檢索多一輪推理。
- **資源成本高**：候選文件越多，計算時間越長。

---

## 最佳實務
- **兩階段檢索**：先快後精——向量檢索 → Cross Encoder 重排。
- **控制候選數量**：重排前 50–100 筆，最後保留 3–5 筆用於生成。
- **批次處理**：一次送多筆文件以提升效率。

---

## 常用模型
- `ms-marco-MiniLM-L-6-v2`：速度與精度平衡。
- `ms-marco-electra-base`：精度更高，適合離線批次。
- 多語言版本：適合跨語言檢索。

---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials.
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
    }
)

You can alternatively set OPENAI_API_KEY in .env file and load it.

[Note] This is not necessary if you've already set OPENAI_API_KEY in previous steps.

In [ ]:
# Configuration file to manage API keys as environment variables
from dotenv import load_dotenv

# Load API key information
load_dotenv(override=True)

In [ ]:
%%capture --no-stderr
%pip install langchain-opentutorial

## 主要功能和機制
### 目的
- 重新排序檢索到的文件以改善其排名，優先顯示與查詢最相關的結果。

### 結構
- 接受**查詢**和**文件**作為單一輸入對，實現聯合處理。

### 機制
- **單一輸入對**：
 將**查詢**和**文件**作為組合輸入處理，直接輸出相關性分數。
- **自注意力機制**：
 使用自注意力聯合分析**查詢**和**文件**，有效捕捉其語義關係。

### 優勢
- **更高準確性**：
 提供更精確的相似性分數。
- **深度上下文分析**：
 探索**查詢**和**文件**間的語義細節。

### 限制
- **高計算成本**：
 處理過程耗時較長。
- **擴展性問題**：
 未經優化不適合大規模文件集合。

---

## 我的見解

Cross Encoder 的核心價值在於語義理解的深度，它透過聯合編碼實現了比傳統向量檢索更精確的相關性評估。

## 學習補充重點

**關鍵技術特點：**
- **聯合編碼**：同時處理查詢和文件，捕捉交互語義
- **精確評分**：輸出 0-1 相關性分數
- **上下文感知**：理解複雜的語義關係

**使用時機：**
- 檢索精度要求高的場景
- 文件數量中等（<100 候選）
- 可接受額外計算成本的應用

**優化策略：**
- 兩階段檢索：先粗排再精排
- 批次處理提升效率
- 模型量化減少計算負擔

## Practical Applications
- A **Bi-Encoder** quickly retrieves candidate **documents** by computing lightweight similarity scores.  
- A **Cross Encoder** refines these results by deeply analyzing the semantic relationship between the **query** and the retrieved **documents** .

## Implementation
- Use Hugging Face cross encoders or ```BAAI/bge-reranker``` models.
- Easily integrate with frameworks like **LangChain** through the ```CrossEncoderReranker``` component.

## 重排器的主要優勢
- **精確相似度評分**：
 提供**查詢**和**文件**之間高度準確的相關性測量。
- **語義深度**：
 分析更深層的語義關係，揭示**查詢**-**文件**互動中的細微差別。
- **改善搜尋品質**：
 提升檢索到的**文件**的相關性和品質。
- **RAG 系統增強**：
 透過改善輸入相關性來增強**檢索增強生成（RAG）**系統的效能。
- **無縫整合**：
 易於適應各種工作流程，與多種框架相容。
- **模型多樣性**：
 提供廣泛的預訓練模型靈活性，滿足客製化需求。

---

## 我的見解

這些優勢清楚展示了重排器在 RAG 系統中的核心價值：透過深度語義分析大幅提升檢索精度。

## 學習補充重點

**核心優勢應用：**
- **精確評分**：解決向量檢索的語義偏差問題
- **品質提升**：顯著改善最終生成內容的相關性
- **易於部署**：與現有系統整合成本低
- **靈活配置**：可根據不同領域選擇專門模型

**實際效益：**
- 檢索準確率提升 15-30%
- 用戶滿意度明顯改善
- 減少無關內容干擾

## Document Count Settings for Reranker
- Reranking is generally performed on the top **5–10** **documents** retrieved during the initial search.
- The ideal number of **documents** for reranking should be determined through experimentation and evaluation, as it depends on the dataset characteristics and computational resources available.

## 使用重排器的權衡考量
- **準確性 vs 處理時間**：
 在實現更高準確性和最小化處理時間之間取得平衡。
- **效能改善 vs 計算成本**：
 權衡效能改善的好處與所需額外計算資源。
- **搜尋速度 vs 相關性準確度**：
 管理更快檢索與維持結果高相關性之間的權衡。
- **系統需求**：
 確保系統滿足支援重排所需的硬體和軟體需求。
- **資料集特性**：
 考慮資料集的規模、多樣性和特定屬性來優化重排器效能。

---

## 我的見解

這些權衡考量反映了實際部署重排器時需要面對的核心決策點，每個組織都需要根據自身需求找到最佳平衡點。

## 學習補充重點

**決策框架：**
- **業務優先級**：效果提升 vs 成本控制
- **用戶體驗**：響應速度 vs 結果品質
- **資源配置**：硬體投資 vs 運營成本
- **規模考量**：小規模精確 vs 大規模效率

**實用建議：**
- 先在小規模測試評估效益
- 使用 A/B 測試量化改善效果
- 考慮混合策略：關鍵查詢用重排器
- 監控系統資源使用情況

**優化策略：**
- 非同步處理降低延遲感知
- 快取熱門查詢結果
- 使用輕量級模型平衡效果與速度

Explaining the Implementation of **Cross Encoder Reranker** with a Simple Example

In [ ]:
# Helper function to format and print document content
def pretty_print_docs(docs):
    # Print each document in the list with a separator between them
    print(
        f"\n{'-' * 100}\n".join(  # Separator line for better readability
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]  # Format: Document number + content
        )
    )

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load documents
documents = TextLoader("./data/appendix-keywords.txt").load()

# Configure text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Split documents into chunks
texts = text_splitter.split_documents(documents)

# # Set up the embedding model
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/msmarco-distilbert-dot-v5",
    model_kwargs = {"tokenizer_kwargs": {"clean_up_tokenization_spaces": False}}
)

# Create FAISS index from documents and set up retriever
retriever = FAISS.from_documents(texts, embeddings_model).as_retriever(
    search_kwargs={"k": 10}
)

# Define the query
query = "Can you tell me about Word2Vec?"

# Execute the query and retrieve results
docs = retriever.invoke(query)

# Display the retrieved documents
pretty_print_docs(docs)

Document 1:

Word2Vec
Definition: Word2Vec is a technique in NLP that maps words to a vector space, representing their semantic relationships based on context.
Example: In a Word2Vec model, "king" and "queen" are represented by vectors located close to each other.
Related Keywords: Natural Language Processing (NLP), Embedding, Semantic Similarity
----------------------------------------------------------------------------------------------------
Document 2:

Token
Definition: A token refers to a smaller unit of text obtained by splitting a larger piece of text. It can be a word, phrase, or sentence.
Example: The sentence "I go to school" can be tokenized into "I," "go," "to," and "school."
Related Keywords: Tokenization, Natural Language Processing (NLP), Syntax Analysis
----------------------------------------------------------------------------------------------------
Document 3:

Example: A customer information table in a relational database is an example of structured data.
Related

現在，讓我們用 ```ContextualCompressionRetriever``` 包裝 ```base_retriever```。```CrossEncoderReranker``` 利用 ```HuggingFaceCrossEncoder``` 來重新排序檢索結果。

---

## 我的見解

這個整合展示了如何將重排器無縫集成到現有的檢索管道中，```ContextualCompressionRetriever``` 作為包裝器提供了優雅的抽象層。

## 學習補充重點

**架構組成：**
- **base_retriever**：基礎檢索器（如向量檢索）
- **ContextualCompressionRetriever**：壓縮和優化包裝器
- **CrossEncoderReranker**：使用 HuggingFace 模型的重排組件

**技術整合優勢：**
- **模組化設計**：可替換不同的基礎檢索器
- **透明介面**：保持原有 API 不變
- **效能優化**：結合檢索速度和排序精度

**實作要點：**
- 選擇合適的 CrossEncoder 模型
- 配置重排候選數量
- 設定相關性閾值過濾

**使用場景：**
- 需要高精度檢索的問答系統
- 複雜查詢的文檔搜尋
- 企業知識管理系統

Multilingual Support BGE Reranker: [```bge-reranker-v2-m3```](https://huggingface.co/BAAI/bge-reranker-v2-m3)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

# Initialize the model
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

# Select the top 3 documents
compressor = CrossEncoderReranker(model=model, top_n=3)

# Initialize the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# Retrieve compressed documents
compressed_docs = compression_retriever.invoke("Can you tell me about Word2Vec?")

# Display the documents
pretty_print_docs(compressed_docs)

Document 1:

Word2Vec
Definition: Word2Vec is a technique in NLP that maps words to a vector space, representing their semantic relationships based on context.
Example: In a Word2Vec model, "king" and "queen" are represented by vectors located close to each other.
Related Keywords: Natural Language Processing (NLP), Embedding, Semantic Similarity
----------------------------------------------------------------------------------------------------
Document 2:

Open Source
Definition: Open source software allows its source code to be freely used, modified, and distributed, fostering collaboration and innovation.
Example: The Linux operating system is a well-known open source project.
Related Keywords: Software Development, Community, Technical Collaboration
Structured Data
Definition: Structured data is organized according to a specific format or schema, making it easy to search and analyze.
--------------------------------------------------------------------------------------------------